<a href="https://colab.research.google.com/github/nitinfaye/Text-classification/blob/main/document_Search_%26_Find_in_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exploratory Data Analysis (EDA)

import the necessary libraries.

In [ ]:
!pip install mglearn

In [ ]:
import os
import csv
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,confusion_matrix
from scipy.sparse import csr_matrix, lil_matrix
#from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data_raw = pd.read_csv('/content/abcnews-date-text.csv')
data_raw =data_raw[:249]
#data_raw = df.drop(['publish_date','tag'], axis=1)
#data_raw =df
data_raw.head()

FileNotFoundError: ignored

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
data_raw["target"] = lb_make.fit_transform(data_raw["tag"])
data_raw[["tag", "target"]].head(11)

In [ ]:
data_raw.shape

In [ ]:
'''def convert_publish_date(publish_date):
  if publish_date <= 20030219:
    out=0
  elif publish_date==20030220:
    out=1
  else:
    out=2
  return out'''

In [ ]:
'''data_raw['target'] = data_raw['publish_date'].apply(lambda x: convert_publish_date(x))
data_raw =data_raw[:249]
data_raw.head()'''

In [ ]:
data_raw = data_raw.drop('publish_date', axis= 1)
print("Number of rows in data =",data_raw.shape[0])
print("Number of columns in data =",data_raw.shape[1])
print("\n")
print("**Sample data:**")
data_raw.head()


In [ ]:
data_raw['tag']= data_raw['tag'].replace(0)
data_raw.dropna(inplace=True)
missing_values_check = data_raw.isnull().sum()
print(missing_values_check)

In [ ]:
# Comments with no label are considered to be clean comments.
# Creating seperate column in dataframe to identify clean comments.

# We use axis=1 to count row-wise and axis=0 to count column wise

rowSums = data_raw.iloc[:,2:].sum(axis=1)
clean_comments_count = (rowSums==1).sum(axis=0)

print("Total number of comments = ",len(data_raw))
print("Number of clean comments = ",clean_comments_count)
print("Number of comments with labels =",(len(data_raw)-clean_comments_count))

In [ ]:
categories = list(data_raw.columns.values)
categories = categories[2:]
print(categories)

In [ ]:
counts = []
for category in categories:
    counts.append((category, data_raw[category].sum()))
df_stats = pd.DataFrame(counts, columns=['category', 'number of comments'])
df_stats

In [ ]:
sns.set(font_scale = 2)
plt.figure(figsize=(15,8))

ax= sns.barplot(categories, data_raw.iloc[:,2:].sum().values)

plt.title("Comments in each category", fontsize=24)
plt.ylabel('Number of comments', fontsize=18)
plt.xlabel('Comment Type ', fontsize=18)

#adding the text labels
rects = ax.patches
labels = data_raw.iloc[:,2:].sum().values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom', fontsize=10)

plt.show()


In [ ]:
rowSums = data_raw.iloc[:,2:].sum(axis=1)
multiLabel_counts = rowSums.value_counts()
#multiLabel_counts = multiLabel_counts.iloc[1:]

sns.set(font_scale = 2)
plt.figure(figsize=(15,8))

ax = sns.barplot(multiLabel_counts.index, multiLabel_counts.values)

plt.title("Comments having multiple labels ")
plt.ylabel('Number of comments', fontsize=18)
plt.xlabel('Number of labels', fontsize=18)

#adding the text labels
rects = ax.patches
labels = multiLabel_counts.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

plt.show()

WordCloud representation of most used words in each category of comments

In [ ]:
from wordcloud import WordCloud,STOPWORDS

plt.figure(figsize=(40,25))

# Headline_text
#subset = data_raw[data_raw.target==0]
text = data_raw.headline_text.values
cloud_headline = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))

plt.subplot(2, 3, 1)
plt.axis('off')
plt.title("Headline_text",fontsize=40)
plt.imshow(cloud_headline)

# tag
#subset = data_raw[data_raw.tag==0]
text = data_raw.tag.values
cloud_tag = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))

plt.subplot(2, 3, 2)
plt.axis('off')
plt.title("tag",fontsize=40)
plt.imshow(cloud_tag)


# Target
subset = data_raw[data_raw.target==2]
text = data_raw.headline_text.values
cloud_headline = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))

plt.subplot(2, 3, 3)
plt.axis('off')
plt.title("Target",fontsize=40)
plt.imshow(cloud_headline)
plt.show()


Data Pre-Processing

In [ ]:
data = data_raw
data = data_raw.loc[np.random.choice(data_raw.index, size=249)]
data.shape

Cleaning Data

In [ ]:
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext


def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned


def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

In [ ]:
data['headline_text'] = data['headline_text'].str.lower()
data['headline_text'] = data['headline_text'].apply(cleanHtml)
data['headline_text'] = data['headline_text'].apply(cleanPunc)
data['headline_text'] = data['headline_text'].apply(keepAlpha)
data.head()

Removing Stop Words

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

data['headline_text'] = data['headline_text'].apply(removeStopWords)
#data['tag'] = data['tag'].apply(removeStopWords)
data.head()

Stemming

In [ ]:
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

data['headline_text'] = data['headline_text'].apply(stemming)
#data['tag'] = data['tag'].apply(stemming)
data.head()

#Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, random_state=42, test_size=0.30, shuffle=True)

print(train.shape)
print(test.shape)

In [ ]:
train_text = train['headline_text']
test_text = test['headline_text']


In [ ]:
train_text

#TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(train_text)
vectorizer.fit(test_text)

In [ ]:
x_train = vectorizer.transform(train_text)
y_train = train.drop(labels = ['headline_text','tag'], axis=1)
#y_train = train(labels= ['target'],axis=1)

x_test = vectorizer.transform(test_text)
y_test = test.drop(labels = ['headline_text','tag'], axis=1)
#y_test =test(labels= ['target'],axis =1)


In [ ]:
x_train

# Classification 

In [ ]:
%%time

# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

for category in categories:
    printmd('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print("\n")

In [ ]:
x_train = lil_matrix(x_train).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(x_test).toarray()


In [ ]:
x_train

##gaussian naive bayes base allgoritm

In [ ]:
%%time

# with a gaussian naive bayes base classifier
classifier = GaussianNB()

# train
classifier.fit(x_train, y_train)

# predict
prediction = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,prediction))
print("\n")
print("Training set score: {:.2f}".format(classifier.score(x_train, y_train)))
print("Testing set score: {:.2f}".format(classifier.score(x_test, y_test)))
print("\n")
#print(classification_report(y_test,prediction))

Classifier Chains

Multi-label classificattion 

In [ ]:
# using classifier chains
from sklearn.linear_model import LogisticRegression

In [ ]:
%%time

# initialize classifier chains multi-label classifier
classifier= LogisticRegression()

# Training logistic regression model on train data
classifier.fit(x_train, y_train)

# predict
prediction = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,prediction))
print("\n")
print("Training set score: {:.2f}".format(classifier.score(x_train, y_train)))
print("Testing set score: {:.2f}".format(classifier.score(x_test, y_test)))
print("\n")
#print(classification_report(y_test,prediction))

LogisticRegression alogorithms

In [ ]:
%%time

# initialize label powerset multi-label classifier
classifier = LogisticRegression()

# train
classifier.fit(x_train, y_train)

# predict
prediction = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,prediction))
print("\n")
print("Training set score: {:.2f}".format(classifier.score(x_train, y_train)))
print("Testing set score: {:.2f}".format(classifier.score(x_test, y_test)))
print("\n")
#print(classification_report(y_test,prediction))

##KNeighborsClassifier Algorithm

#knn

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors = 1, metric = 'minkowski', p = 2)
classifier.fit(x_train, y_train)

prediction = classifier.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,prediction))
print("\n")
print("Training set score: {:.2f}".format(classifier.score(x_train, y_train)))
print("Testing set score: {:.2f}".format(classifier.score(x_test, y_test)))
print("\n")
#print(classification_report(y_test,prediction))

In [ ]:
np.sqrt(250)

In [ ]:
%%time
classifier = KNeighborsClassifier(n_neighbors =15, metric = 'minkowski', p = 2)
classifier.fit(x_train, y_train)

prediction = classifier.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,prediction))
print("\n")
print("Training set score: {:.2f}".format(classifier.score(x_train, y_train)))
print("Testing set score: {:.2f}".format(classifier.score(x_test, y_test)))
print("\n")
#print(classification_report(y_test,prediction))



In [ ]:
# KNN Plot
#x_train, y_train = mglearn.make_forge()
mglearn.plots.plot_knn_classification(n_neighbors=1)
mglearn.discrete_scatter(x_train[:, 0], x_train[:, 1], y_train)
plt.legend(["Class 0", "Class 1"], loc=4)
plt.xlabel("First feature")
plt.ylabel("Second feature")
plt.show()

In [ ]:
#KNN plot
#x_train, y_train = mglearn.make_forge()
mglearn.plots.plot_knn_classification(n_neighbors=15)
mglearn.discrete_scatter(x_train[:, 0], x_train[:, 1], y_train)
plt.legend(["Class 0", "Class 1"], loc=4)
plt.xlabel("First feature")
plt.ylabel("Second feature")
plt.show()

#Visualising the Training set results